In [2]:
import pandas as pd    
checkpoints = [
    # "facebook/wav2vec2-large-slavic-voxpopuli-v2",
    # "facebook/wav2vec2-large-960h-lv60-self",
    # "classla/wav2vec2-large-slavic-parlaspeech-hr",
    "facebook/hubert-large-ls960-ft",
    "facebook/hubert-xlarge-ls960-ft",
]
optimal_epochs = {
    "facebook/wav2vec2-large-960h-lv60-self": 9,
    "facebook/wav2vec2-large-slavic-voxpopuli-v2": 11,
    "classla/wav2vec2-large-slavic-parlaspeech-hr": 7,  
    "facebook/hubert-large-ls960-ft": 10,
    "facebook/hubert-xlarge-ls960-ft": 10,
    
}
from utils import train_model, eval_model
from hubertutils import train_model as hu_train_model, eval_model as hu_eval_model
for checkpoint in checkpoints * 10:
    import os
    os.system("rm -r models/*")
    train_config = dict(
        model_name_or_path = checkpoint,
        TASK = "emotion_10_epochs",
        NUM_EPOCH = optimal_epochs.get(checkpoint),
        output_column = "target",
        input_column = "path",
        data_files = {
            "train": "007_train.csv",
            "validation": "007_dev.csv",
        },
        clip_seconds = 10
    )

    output_dir = hu_train_model(train_config)
    import numpy as np
    from pathlib import Path
    found_path = str(list(Path(output_dir).glob("checkpoint-*"))[0])
    for split in "dev test".split():
        results = []
        # print(f"For checkpoint: {checkpoint} a path was found: ", found_path)
        eval_config = dict(
            output_column = "target",
            model_name_or_path= found_path ,
            eval_file= f"007_{split}.csv"
        )

        y_true, y_pred = hu_eval_model(eval_config)
        import datetime
        time = str(datetime.datetime.now())
        results.append(
            {
                **eval_config, 
                **train_config,
                "split": split,
                "y_true": y_true, 
                "y_pred": y_pred,
                "time": time,
                        })
        content = pd.DataFrame(data=results).to_json(None, 
                                           orient="records",
                                           lines=True,
                                           )
        with open("009_hubert_training.jsonl", "a") as f:
            f.writelines(content)

rm: cannot remove 'models/*': No such file or directory
Using custom data configuration default-9260442a7fe0355e
Reusing dataset csv (/home/peterr/.cache/huggingface/datasets/csv/default-9260442a7fe0355e/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

A classification problem with 5 classes: ['anger', 'fear', 'happiness', 'neutral', 'sadness']
The target sampling rate: 16000


/home/peterr/anaconda3/lib/python3.8/site-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/home/peterr/anaconda3/lib/python3.8/site-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  tensor = as_tensor(value)
/home/peterr/anaconda3/lib/python3.8/site-packages/transformers/feature_extraction_utils.py:168: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-

Epoch,Training Loss,Validation Loss,Accuracy
0,1.258600,1.519196,0.415842
1,1.076700,1.690176,0.445545
2,0.612500,1.153252,0.623762
3,0.656300,1.225020,0.683168
4,0.276100,1.310936,0.693069
5,0.624300,1.607441,0.673267
6,0.557800,1.637533,0.707921
7,0.057300,1.784918,0.722772
8,0.215200,1.870361,0.722772
9,0.331000,1.835797,0.737624


...

A classification problem with 5 classes: ['anger', 'fear', 'happiness', 'neutral', 'sadness']


loading configuration file https://huggingface.co/facebook/hubert-large-ls960-ft/resolve/main/config.json from cache at /home/peterr/.cache/huggingface/transformers/6e6a66209ff036c1950c074bf469284b972c51d47a2b5616058492342c5aefed.757c0a4bd6fb6d02fc8d08d3765246a0dfceff43b5c30b6f89a37f5bc41cf9a9
Model config HubertConfig {
  "_name_or_path": "facebook/hubert-large-ls960-ft",
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "architectures": [
    "HubertForCTC"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "conv_bias": true,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "diversity_loss_weight": 0.1,
  "do_stable_layer_norm": true,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_

The target sampling rate: 16000


loading weights file https://huggingface.co/facebook/hubert-large-ls960-ft/resolve/main/pytorch_model.bin from cache at /home/peterr/.cache/huggingface/transformers/3964c36efc75c1687db64aaf1d970beee48cd89e4a8c6b5bd072ca7ee9ff8cc5.bb6f85ae361fc3c32f100ac2335b4609e9f7135ced8e598265e36654b51d8471
Some weights of the model checkpoint at facebook/hubert-large-ls960-ft were not used when initializing HubertForSpeechClassification: ['lm_head.bias', 'lm_head.weight']
- This IS expected if you are initializing HubertForSpeechClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing HubertForSpeechClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of HubertForSpeechClassificati

Epoch,Training Loss,Validation Loss
